Setting Up the Knowledge Graph (Neo4j)

In [ ]:
from neo4j import GraphDatabase

class Neo4jConnector:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
    
    def create_node(self, label, properties):
        with self.driver.session() as session:
            session.run(f"CREATE (n:{label} $props)", props=properties)
    
    def create_relationship(self, from_id, to_id, rel_type):
        with self.driver.session() as session:
            session.run(
                "MATCH (a), (b) WHERE a.id = $from_id AND b.id = $to_id "
                f"CREATE (a)-[:{rel_type}]->(b)",
                from_id=from_id, to_id=to_id
            )

# Initialize connection
neo4j = Neo4jConnector("bolt://localhost:7687", "collins", "*******")

Generating Test Cases Using OpenAI API

In [ ]:
import openai

def generate_test_case(requirement):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "Generate test cases for the software requirement."},
            {"role": "user", "content": f"Requirement: {requirement}"}
        ]
    )
    return response.choices[0].message['content']

# Example usage
requirement = "User login must authenticate via OAuth2."
test_case = generate_test_case(requirement)
print(test_case)

Coverage validation Using Knowledge Graph

In [ ]:
def validate_coverage(test_case_id, requirement_id):
    with neo4j.driver.session() as session:
        result = session.run(
            "MATCH (t:TestCase {id: $test_case_id})-[:VALIDATES]->(r:Requirement {id: $requirement_id}) "
            "RETURN t IS NOT NULL AS is_valid",
            test_case_id=test_case_id, requirement_id=requirement_id
        )
        return result.single()["is_valid"]

# Example validation
is_valid = validate_coverage("TC1", "REQ1")
print(f"Test case covers requirement: {is_valid}")

Workflow Integration

In [ ]:
def end_to_end_workflow(requirement_desc, req_id):
    # Generate test case
    test_case = generate_test_case(requirement_desc)
    # Store in knowledge graph
    neo4j.create_node("TestCase", {"id": "TC1", "steps": test_case})
    neo4j.create_relationship("TC1", req_id, "VALIDATES")
    # Validate
    return validate_coverage("TC1", req_id)

# Example
result = end_to_end_workflow("Payment gateway must support HTTPS.", "REQ2")
print(f"Coverage valid: {result}")